# Fundamentals of Machine Learning - Exercise 9
Goal of this excercise is to complete the hands-on experience of the classification task.

## Household Prices Dataset
https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data

* ... I bet that you already know the data pretty well 😅

![meme03](https://github.com/rasvob/VSB-FEI-Fundamentals-of-Machine-Learning-Exercises/blob/master/images/fml_09_meme_03.jpg?raw=true)

**Important attributes description:**
* SalePrice: The property's sale price in dollars. This is the target variable that you're trying to predict.
* MSSubClass: The building class
* BldgType: Type of dwelling
* HouseStyle: Style of dwelling
* OverallQual: Overall material and finish quality
* OverallCond: Overall condition rating
* YearBuilt: Original construction date
* Heating: Type of heating
* CentralAir: Central air conditioning
* GrLivArea: Above grade (ground) living area square feet
* BedroomAbvGr: Number of bedrooms above basement level)



In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix, auc
from sklearn.preprocessing import OrdinalEncoder

# 🎯 Our goal is to predict if the house will be sold for more than 250k USD or not
* We will use categorized price as a **Target** variable

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/rasvob/VSB-FEI-Fundamentals-of-Machine-Learning-Exercises/master/datasets/zsu_cv1_data.csv', sep=',')
df = df.loc[:, ['SalePrice','MSSubClass','BldgType','HouseStyle','OverallQual','OverallCond','YearBuilt','Heating','CentralAir','GrLivArea','BedroomAbvGr']]
df.loc[:, ['Target']] = (df.SalePrice > 250000).astype(int)
df = df.drop(['SalePrice'], axis=1)

In [3]:
df.head()

MSSubClass BldgType HouseStyle  OverallQual  OverallCond  YearBuilt  \
0          60     1Fam     2Story            7            5       2003   
1          20     1Fam     1Story            6            8       1976   
2          60     1Fam     2Story            7            5       2001   
3          70     1Fam     2Story            7            5       1915   
4          60     1Fam     2Story            8            5       2000   

  Heating CentralAir  GrLivArea  BedroomAbvGr  Target  
0    GasA          Y       1710             3       0  
1    GasA          Y       1262             3       0  
2    GasA          Y       1786             3       0  
3    GasA          Y       1717             3       0  
4    GasA          Y       2198             4       0

# Take a look at the features
* We will need it to answer the questions

In [4]:
df.describe()

MSSubClass  OverallQual  OverallCond    YearBuilt    GrLivArea  \
count  1460.000000  1460.000000  1460.000000  1460.000000  1460.000000   
mean     56.897260     6.099315     5.575342  1971.267808  1515.463699   
std      42.300571     1.382997     1.112799    30.202904   525.480383   
min      20.000000     1.000000     1.000000  1872.000000   334.000000   
25%      20.000000     5.000000     5.000000  1954.000000  1129.500000   
50%      50.000000     6.000000     5.000000  1973.000000  1464.000000   
75%      70.000000     7.000000     6.000000  2000.000000  1776.750000   
max     190.000000    10.000000     9.000000  2010.000000  5642.000000   

       BedroomAbvGr       Target  
count   1460.000000  1460.000000  
mean       2.866438     0.148630  
std        0.815778     0.355845  
min        0.000000     0.000000  
25%        2.000000     0.000000  
50%        3.000000     0.000000  
75%        3.000000     0.000000  
max        8.000000     1.000000

## Categorial features EDA

In [5]:
df.describe(exclude=np.number)

BldgType HouseStyle Heating CentralAir
count      1460       1460    1460       1460
unique        5          8       6          2
top        1Fam     1Story    GasA          Y
freq       1220        726    1428       1365

### BldgType

In [6]:
df.BldgType.value_counts()

BldgType
1Fam      1220
TwnhsE     114
Duplex      52
Twnhs       43
2fmCon      31
Name: count, dtype: int64

### HouseStyle

In [7]:
df.HouseStyle.value_counts()

HouseStyle
1Story    726
2Story    445
1.5Fin    154
SLvl       65
SFoyer     37
1.5Unf     14
2.5Unf     11
2.5Fin      8
Name: count, dtype: int64

### Heating

In [8]:
df.Heating.value_counts()

Heating
GasA     1428
GasW       18
Grav        7
Wall        4
OthW        2
Floor       1
Name: count, dtype: int64

## Missing values

In [9]:
df.isna().sum()

MSSubClass      0
BldgType        0
HouseStyle      0
OverallQual     0
OverallCond     0
YearBuilt       0
Heating         0
CentralAir      0
GrLivArea       0
BedroomAbvGr    0
Target          0
dtype: int64

## Labels distribution

In [10]:
df.Target.value_counts()

Target
0    1243
1     217
Name: count, dtype: int64

# ✅ Task (2p)
Complete the following tasks:

1. 📈 Describe what operations you are performing for each of the features
    * Mainly focus on categorical features
      
2. 📌 Answer the following questions:
    * **How many values are missing?**
    * **How many instances do you have in each of the classes?**
    * 🔎 **Which metric score do you propose for the classification model performance evaluation?**
        * 💡 This depends on your previous answer
          
3. ⚡Finish your preprocessing pipeline and split the data into the Input and Output part (i.e. `X` and `y` variables)

4. 🌳 Start with the **Decision Tree**
    * Use 5-fold cross validation
    * 🔎 Will you use *standard* cross validation or *stratified* cross validation?
    * Compute mean of the obtained score values
      
5. 🚀 Select one other algorithm from https://scikit-learn.org/stable/supervised_learning.html
    * Repeat the 5-fold CV
      
6. 📒 **Write down which default model is better**

7. 📊 Experiment with hyper-parameters
    * Select at least one important parameter for the model 
    * Set the parameter value range
        * You can use random values, interval of values, ... 
    * Do the 5-fold CV
        * Compute mean of the obtained score values
    * Document the experiment results using tables and/or plots
    * Describe the results in a Markdown cell

8. 📒 **Write down which model (default or tuned) is the best and why**

* **Document everything you do in a Markdown cells**
    * ❌ Results interpretation figured in real-time during task check is not allowed! ❌

![meme01](https://github.com/rasvob/VSB-FEI-Fundamentals-of-Machine-Learning-Exercises/blob/master/images/fml_09_meme_01.jpg?raw=true)

In [ ]:
x, y = df.loc[:, df.columns != 'Target'], df.Target
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
